<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Tracing Smolagents with Arize Phoenix</h1>

## Install Dependencies

In [ ]:
!pip install smolagents -q
!pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

## Connect to Phoenix


In this example, we'll use Phoenix as our destination. You could instead add any other exporters you'd like in this approach.

If you need to set up an API key for Phoenix, you can do so [here](https://app.phoenix.arize.com/).

#### Keys

Note: For this colab you'll need:

*   Phoenix API key
*   Hugging Face Token

In [ ]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass.getpass("Enter your Phoenix API Key: ")
if not (hf_token_value := os.getenv("HF_TOKEN")):
    hf_token_value = getpass.getpass("Enter your Hugging Face Token: ")

# Set the environment variables with the provided keys
os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={phoenix_api_key}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"
os.environ["HF_TOKEN"] = f"{hf_token_value}"

### Define the Trace Provider

In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="my-agent-app",  # Default is 'default'
)

## Instrument Smolagents




In [ ]:
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

SmolagentsInstrumentor().instrument(tracer_provider=tracer_provider)

## Create your Smolagents & Run


Now you can create any sort of agent using the smolagents module, and Phoenix will track every step, input, output, tool, and call your agent makes.

In [ ]:
from smolagents import (
    CodeAgent,
    DuckDuckGoSearchTool,
    HfApiModel,
    ManagedAgent,
    ToolCallingAgent,
    VisitWebpageTool,
)

model = HfApiModel()

agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
)
managed_agent = ManagedAgent(
    agent=agent,
    name="managed_agent",
    description="This is an agent that can do web search.",
)
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[managed_agent],
)

manager_agent.run("What are Smolagents?")

 ## ⭐⭐⭐ If you like this guide, please give [Arize Phoenix](https://github.com/Arize-ai/phoenix) a star ⭐⭐⭐